# API Communication Channels

---

When communicating with the GridAPPS-D Platform through API, it is necessary to specify a communication channel, which tells the GridAPPS-D platform on which channel to communicate with the application and through which API the message should be directed.

---

## `/queue/` vs `/topic/` Channels

GridAPPS-D uses two types of communication channels to determine the visibility of the API call to other applications and services.

### Queue Channels

`/queue/` is used for communication channels where only the GridAPPS-D Platform is listening to the API call. These API calls are processed on a first-in, first-out basis. There is only one subscriber to the communication channel.

API calls to the Blazegraph database, Logs, Timeseries database, Config files, and Platform status are all queue channels. All the GridAPPS-D Topics for queue channels typically do not change over the course of an application or simulation run.

In the GridAPPSD-Python library, it is assumed that a topic is a queue channel if not otherwise specified. These two GridAPPS-D Topic definitions are equivalent:

`topic = '/queue/goss.gridappsd.process.request.data.powergridmodel'`

`topic = 'goss.gridappsd.process.request.data.powergridmodel'`


### Topic Channels

`/topic/` is used for communication channels where the API call is to broadcast to all subscribers through the GOSS Message Bus, inlcuding other applications, services, FNCS Bridge, etc. 

API calls to the Simulation, services, and active applications use topic channels to communicate and typically need to the specify the Simulation IDs, Service IDs, and Application IDs. The particular topic for such an API call will change between simulations and instances, and so shortcut functions are provided in GridAPPSD-Python library to assist in generating the correct Topic.

In GridAPPSD-Python, it is necessary to specify if a GridAPPS-D Topic is a `/topic/` channel broadcasting to all subscribers:

`topic = "/topic/goss.gridappsd.simulation.input."+simulation_id`

---

## Static GridAPPS-D Topics

Below are a list of the most common topics and where they are used. The appropriate topic for each API call will also be listed again in the subsequent sections on each GridAPPS-D API. The list below can serve as an additional convenient reference.

These topics remain the remain the same between platform, application, and simulation instances. The GridAPPSD-Python Library shortcuts use all uppercase naming to indicate that these are static topic names.

### Importing the Topics Library

When using topics in GridAPPSD-Python, it is recommended to import the `topics` library from `gridappsd`. This enables you to rapidly call the correct topic without needing to search for the correct topic string. This also protects your code from any changes inside the GridAPPS-D Platform if particular topic strings are deprecated or replaced -- the python library names will stay persistent between all Platform releases.

For static GridAPPS-D topics, import the library by running

In [ ]:
from gridappsd import topics as t

### Request PowerGrid Model Data

This `/queue/` channel is used to communicate with PowerGrid Models API to pull power system model info from the the Blazegraph Database. The PowerGrid Model API is covered in detail in [Using the PowerGrid Models API](../api_usage/3.3-Using-the-PowerGrid-Models-API.ipynb).

The base static string used is `goss.gridappsd.process.request.data.powergridmodel`, which can be called using the `.REQUEST_POWERGRID_DATA` or `.BLAZEGRAPH` methods from the topics library

A sample message that would be passed with this topic is 

In [ ]:
from gridappsd import topics as t

# Sample PowerGrid Model message
message = '{"requestType": "QUERY_MODEL_NAMES", "resultFormat": "JSON"}'

gapps.get_response(t.REQUEST_POWERGRID_DATA, message)

In [ ]:
from gridappsd import topics as t

# Sample PowerGrid Model message
message = '{"requestType": "QUERY_MODEL_NAMES", "resultFormat": "JSON"}'

gapps.get_response(t.BLAZEGRAPH, message)

---

### Request Timeseries Data

This `/queue/` channel is used to communicate with the Timeseries API and Timeseries database, which stores real-time and historical data, such as weather information and AMI meter readings. The Timeseries database is covered in detail in [Using the Timeseries API](../api_usage/3.7-Using-the-Timeseries-API.ipynb). A sample message that would be passed with this topic is

__Text String:__ The topic can be specified as a static string:

* `topic = "goss.gridappsd.process.request.data.timeseries"`
* `gapps.get_response(topic, message)`

__GridAPPSD-Python Library Method:__ The correct topic can also be imported from the GridAPPSD-Python topics library:

* `from gridappsd import topics as t`
* `gapps.get_response(t.TIMESERIES, message)`

---

### Request Platform Status

This topic is used to check that status of the GridAPPS-D Platform. 



__Text String:__ The topic can be specified as a static string:

* `topic = "/queue/goss.gridappsd.process.request.status.platform"`
* `gapps.get_response(topic, message)`

__GridAPPSD-Python Library Method:__ The correct topic can also be imported from the GridAPPSD-Python topics library. 

* `from gridappsd import topics as t`
* `gapps.get_response(t.PLATFORM_STATUS, message)`

---

### Querying Log Data

This topic is used to query log data in the MySQL Database using the Logging API

__Note:__ This topic is different from the one used to subscribe to real-time log data being published by an ongoing simulation. This topic is used for querying data already stored in the database.

__Text String:__ The topic can be specified as a static string:

* `topic = "goss.gridappsd.process.request.data.log"`
* `gapps.get_response(topic, message)`

__GridAPPSD-Python Library Method:__ The correct topic can also be imported from the GridAPPSD-Python topics library:

* `from gridappsd import topics as t`
* `gapps.get_response(t.LOGS, message)`

---

### Subscribing to Platform Logs

This topic is used to subscribe the to logs created by the GridAPPS-D Platform, such as which managers and core services have been started and are running.

__Text String:__ The topic can be specified as a static string:

* `topic = "goss.gridappsd.process.request.data.timeseries"`
* `gapps.get_response(topic, message)`


__GridAPPSD-Python Library Function:__ The correct topic can also be imported from the GridAPPSD-Python topics library. Note that this is a python function similar to the dynamic topics presented in the next section.

* `from gridappsd.topics import platfor_log_topic
* `topic = platform_log_topic()`
* `gapps.get_response(topic, message)`

[[Return to Top](#Table-of-Contents)]

---

## Dynamic GridAPPS-D Topics

Several GridAPPS-D topics are unique to each application, simulation, or service instance. These topics are dynamic and will change from instance to instance. 

The GridAPPS-D Platform will require that the topic specify the particular instance so that the API call can be delivered to the correct simulation or service. 

To assist with the task of creating a dynamic topic that automatically updates between instances, several function are available in the GridAPPSD-Python topics library.

The available GridAPPSD-Python functions for dynamic topics are

* `simulation_input_topic(simulaton_id)` -- Gets the topic to write data to for the simulation


* `simulation_output_topic(simulation_id)` -- Gets the topic for subscribing to output from the simulation


* `simulation_log_topic(simulation_id)` -- Topic for the subscribing to the logs from the simulation


* `service_input_topic(service_id, simulation_id)` -- Utility method for getting the input topic for a specific service


* `service_output_topic(service_id, simulation_id)` -- Utility method for getting the output topic for a specific service


* `application_input_topic(application_id, simulation_id)` -- Utility method for getting the input topic for a specific application


* `application_output_topic(application_id, simulation_id)` -- Utility method for getting the output topic for a specific application


---

### Subscribe to Simulation Output

This topic is used to communicate with the Simulation API, which is covered in detail in [Controlling Simulations with Simulation API](../api_usage/3.6-Controlling-Simulation-API.ipynb). The Simulation Output Topic is used to subscribe to the simulation output, enabling applications to listen to switching actions, obtain equipment measurements, and so on. 

The GridAPPSD-Python shortcut function for generating the correct topic is

`simulation_output_topic(simulation_id)`

There are two ways to use the function. The first is to call the library function directly. The second is to use it as part of a class definition.

__1) Call the topic function directly__

In [ ]:
# Import GridAPPS-D Topic Function:
from gridappsd.topics import simulation_output_topic

# Call GridAPPSD-Python Topic Function
topic = simulation_output_topic(simulation_id)

# Print to Notebook Kernel:
print(topic)

__2) Use the topic function in a class definition__

In [ ]:
# Import GridAPPS-D Topic Function:
from gridappsd.topics import simulation_output_topic

# Define Subscription Class
class MySubscription(object):
    def __init__(self,simulation_id):
        self._subscribe_to_topic = simulation_output_topic(simulation_id)
        
# Define Main Function: 
def _main():
    subscription = MySubscription(simulation_id)
    print(subscription._subscribe_to_topic)
    
# Call Main Function:
_main()

---

### Publish to Simulation Input

This topic is used to communicate with the Simulation API, which is covered in detail in [Controlling Simulations with Simulation API](../api_usage/3.6-Controlling-Simulation-API.ipynb). The Simulation Input Topic is used to publish commands to the GOSS Message Bus, which are then broadcast to all applications, services, and simulations that are listening. Examples of actions that will use this topic include taking switching actions, adjusting DER setpoints, and changing regulator taps.

The GridAPPSD-Python shortcut function for generating the correct topic is

`simulation_input_topic(simulation_id)`

There are two ways to use the function. The first is to call the library function directly. The second is to use it as part of a class definition.

__1) Call the topic function directly__

In [ ]:
# Import GridAPPS-D Topic Function:
from gridappsd.topics import simulation_input_topic

# Call GridAPPSD-Python Topic Function
topic = simulation_output_topic(simulation_id)

# Print to Notebook Kernel:
print(topic)

__2) Use the topic function in a class definition__

In [ ]:
# Import GridAPPS-D Topic Function:
from gridappsd.topics import simulation_input_topic

# Define Subscription Class
class MySimulationPublisher(object):
    def __init__(self,simulation_id):
        self._publish_to_topic = simulation_input_topic(simulation_id)
        
# Define Main Function: 
def _main():
    subscription = MySimulationPublisher(simulation_id)
    print(subscription._publish_to_topic)
    
# Call Main Function:
_main()

---

### Subscribe to Simulation Logs

This topic is used to communicate with the Simulation API, which is covered in detail in Lesson XX. The Simulation Output Topic is used to subscribe to the simulation output, which applications use to
* Listen to switching actions
* Obtaining equipment measurements
* ___GET FULL LIST___

The GridAPPSD-Python shortcut function for generating the correct topic is

`simulation_output_topic(simulation_id)`

There are two ways to use the function. The first is to call the library function directly. The second is to use it as part of a class definition.

__1) Call the topic function directly__

In [ ]:
# Import GridAPPS-D Topic Function:
from gridappsd.topics import simulation_output_topic

# Call GridAPPSD-Python Topic Function
topic = simulation_output_topic(simulation_id)

# Print to Notebook Kernel:
print(topic)

__2) Use the topic function in a class definition__

In [ ]:
# Import GridAPPS-D Topic Function:
from gridappsd.topics import simulation_output_topic

# Define Subscription Class
class MySubscription(object):
    def __init__(self,simulation_id):
        self._subscribe_to_topic = simulation_output_topic(simulation_id)
        
# Define Main Function: 
def _main():
    subscription = MySubscription(simulation_id)
    print(subscription._subscribe_to_topic)
    
# Call Main Function:
_main()

---

![GridAPPS-D-narrow.png](../images/GridAPPS-D_narrow.png)